In [1]:
#预测值
# -*- coding: utf-8 -*-
import copy
import re

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
#import shap
import pickle
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# import xgboost
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.neural_network import MLPClassifier
import catboost
from catboost import *
import warnings
warnings.filterwarnings("ignore")

'''
用五折交叉验证的时候原数据集需打乱顺序
'''
# cd = 'D:/桌面/深圳心脏骤停/患者-学员/宝安区new/整体/'
# cd1 = 'D:/桌面/深圳心脏骤停/患者-学员/宝安区new/整体/y_病人结局/'

rawdata = pd.read_excel('dataset/OHCA_M2.xlsx')#
## 数据集打乱顺序
rawdata = rawdata.sample(frac=1, random_state=42)
# print(rawdata.columns)
# datay= rawdata['Return of spontaneous circulation on-site (M1)']#Patient outcome Return of spontaneous circulation on-site
datay= rawdata['Patient outcome (M2)']#Patient outcome Return of spontaneous circulation on-site

# print(datay)
# print(rawdata['旁观者是否使用AED'].value_counts(dropna=True))
# print(rawdata['是否院外电除颤'].value_counts(dropna=True))

## x变量 (先用几个变量,后面改成所有x变量)
x_features =['age','Bystander use of AEDs','Time to ambulance arrival','Performer of defibrillation_Medical staff',
'Bystander CPR','Initial rhythm of cardiac arrest_Normal heart rhythm',
'Initial rhythm of cardiac arrest_Shockable heart rhythm','Initial rhythm of cardiac arrest_Non-shockable heart rhythm',
'Out-of-hospital electrical defibrillation','Location_Family house','5-minute social rescue circle','Training rate_6 months','Number of AEDs within 75m',
'Use of electrical defibrillation in ED','Use of mechanical CPR device in ED', 'Establishment of advanced artificial airway in ED',
'Use of medications in ED','PCI for ED','TTM for ED','ECMO for ED','Return of spontaneous circulation in ED' 
]

#加上以下特征为M2
#'Use of electrical defibrillation in ED'	'Use of mechanical CPR device in ED'	'Establishment of advanced artificial airway in ED'	
#'Use of medications in ED'	'PCI for ED'	'TTM for ED'	'ECMO for ED'	'Return of spontaneous circulation in ED' 
datax = rawdata[x_features]

# 结果路径
save_path ='M2_compare'
if not os.path.exists(save_path):
    os.makedirs(save_path)
if not os.path.exists(save_path + '/models_compare_results.txt'):
    with open(save_path + '/models_compare_results.txt', "w") as file:
        pass
log = open(save_path + '/models_compare_results.txt', mode="a+", encoding="utf-8")

def fillna_mean(X):
    for column in list(X.columns[X.isnull().sum()>0]):
        mean_val = X[column].mean()
        # mean_val = X[column].median()
        X[column].fillna(mean_val,inplace=True)
    return X


def model_10cv_Catboost(X,y):  ## 10

    name = 'M2'
    fold = 0
    skf = StratifiedKFold(n_splits=5)
    model = CatBoostClassifier(iterations=500, learning_rate=0.01)#, random_seed=12
    ## note X,y is dataFrame, not array, so .split(X,y) is not right
    t = y.values
    allAUC = []
    allY = []
    allPY = []
    for train_index, test_index in skf.split(np.zeros(len(t)),t):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train,y_train)
        probs = model.predict_proba(X_test)
        y_tepred = probs[:,1]
        auc_te = metrics.roc_auc_score(y_test, y_tepred)
        print('auc_te = ', auc_te)
        allAUC.append(auc_te)
        allY.extend(y_test)
        allPY.extend(y_tepred)
        
        fold += 1
        model_save_path = save_path + '/models' + name + '/'
        if not os.path.exists(model_save_path):
            os.makedirs(model_save_path)
        joblib.dump(model, model_save_path + f'catboost_model_fold_{fold}.pkl')

    meanAUC = np.mean(allAUC)
    print('====Catboost model: meanAUC={}'.format(meanAUC))
    print('====Catboost model: 95% CI=[{}, {}]'.format(np.percentile(allAUC, 2.5), np.percentile(allAUC, 97.5)))
    print('====Catboost model: meanAUC={}'.format(meanAUC), file=log)
    print('====Catboost model: 95% CI=[{}, {}]'.format(np.percentile(allAUC, 2.5), np.percentile(allAUC, 97.5)),
          file=log)
    print('====Catboost model: AUCS={}'.format(allAUC), file=log)

    predict_result = pd.DataFrame({'y_test':allY,'y_tepred':allPY})
    if os.path.exists(save_path+'/Catboost/')==False:
        os.makedirs(save_path+'/Catboost/')
    predict_result.to_csv(save_path+'/Catboost/predict_result_Catboost.csv',index=False)
    return predict_result

predict_result = model_10cv_Catboost(datax,datay)


0:	learn: 0.6689580	total: 150ms	remaining: 1m 14s
1:	learn: 0.6479498	total: 152ms	remaining: 37.8s
2:	learn: 0.6336418	total: 154ms	remaining: 25.4s
3:	learn: 0.6044308	total: 158ms	remaining: 19.5s
4:	learn: 0.5780108	total: 162ms	remaining: 16.1s
5:	learn: 0.5525429	total: 167ms	remaining: 13.8s
6:	learn: 0.5270456	total: 173ms	remaining: 12.2s
7:	learn: 0.5070469	total: 177ms	remaining: 10.9s
8:	learn: 0.4899069	total: 181ms	remaining: 9.88s
9:	learn: 0.4748137	total: 183ms	remaining: 8.98s
10:	learn: 0.4588652	total: 186ms	remaining: 8.26s
11:	learn: 0.4434817	total: 188ms	remaining: 7.64s
12:	learn: 0.4238149	total: 190ms	remaining: 7.12s
13:	learn: 0.4062504	total: 192ms	remaining: 6.67s
14:	learn: 0.3931280	total: 194ms	remaining: 6.28s
15:	learn: 0.3778288	total: 196ms	remaining: 5.94s
16:	learn: 0.3662852	total: 198ms	remaining: 5.63s
17:	learn: 0.3516215	total: 200ms	remaining: 5.36s
18:	learn: 0.3410526	total: 202ms	remaining: 5.11s
19:	learn: 0.3276232	total: 204ms	remain